In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

connection_string = f"group5:Group5Project@nbastats2.c4lr0vlcqzd5.us-east-1.rds.amazonaws.com:5432/NBA_Stats"
engine = create_engine(f'postgresql://{connection_string}')

In [3]:
mvp = pd.read_csv("Data/AllPlayerData.csv")
mvp = mvp.drop("Table Name", axis = 1)
mvp = mvp.drop("Rk", axis = 1)
mvp_2020 = mvp.loc[mvp["Season"] == "2019-20"]
mvp = mvp.loc[mvp["Season"] != "2019-20"]
mvp_2019 = mvp.loc[mvp["Season"] == "2018-19"]
mvp_2018 = mvp.loc[mvp["Season"] == "2017-18"]
mvp_2017 = mvp.loc[mvp["Season"] == "2016-17"]

In [4]:
mvp.columns

Index(['Player', 'PlayerID', 'Pos', 'Season', 'Tm', '2P', '2P%', '2PA', '3P',
       '3P%', '3PA', 'Age', 'AST', 'BLK', 'DRB', 'eFG%', 'FG', 'FG%', 'FGA',
       'FT', 'FT%', 'FTA', 'G', 'GS', 'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV',
       'TRB', 'MVP'],
      dtype='object')

In [5]:
mvp_votes = pd.read_csv("Data/MVP.csv")
mvp_votes = mvp_votes[["Player", "Season"]]
mvp_votes["MVP_Candidate"] = 1
mvp_votes

,Player,Season,MVP_Candidate
0,Allen Iverson,2000-01,1
1,Tim Duncan,2000-01,1
2,Shaquille O'Neal,2000-01,1
3,Chris Webber,2000-01,1
4,Kevin Garnett,2000-01,1
...,...,...,...
266,Kevin Durant,2018-19,1
267,Kawhi Leonard,2018-19,1
268,Russell Westbrook,2018-19,1
269,Rudy Gobert,2018-19,1


In [6]:
mvp = pd.merge(mvp, mvp_votes, how='left', on=["Player", "Season"])
mvp["MVP_Candidate"] = mvp["MVP_Candidate"].fillna(0)

In [7]:
mvp_2020 = pd.merge(mvp_2020, mvp_votes, how='left', on=["Player", "Season"])
mvp_2020

,Player,PlayerID,Pos,Season,Tm,2P,2P%,2PA,3P,3P%,...,GS,MP,ORB,PF,PTS,STL,TOV,TRB,MVP,MVP_Candidate
0,Steven Adams,adamsst01,C,2019-20,OKC,261,0.593,440,1,0.333,...,58,1564,196,111,633,50,86,543,0,NaN
1,Bam Adebayo,adebaba01,PF,2019-20,MIA,407,0.576,706,1,0.077,...,65,2235,165,164,1053,78,185,683,0,NaN
2,LaMarcus Aldridge,aldrila01,C,2019-20,SAS,330,0.519,636,61,0.389,...,53,1754,103,128,1001,36,74,392,0,NaN
3,Nickeil Alexander-Walker,alexani01,SG,2019-20,NOP,37,0.336,110,40,0.342,...,0,501,8,46,211,11,40,80,0,NaN
4,Grayson Allen,allengr01,SG,2019-20,MEM,47,0.534,88,34,0.370,...,0,520,5,38,226,8,25,68,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,Thaddeus Young,youngth01,PF,2019-20,CHI,190,0.501,379,79,0.356,...,16,1591,94,134,659,92,102,315,0,NaN
634,Trae Young,youngtr01,PG,2019-20,ATL,341,0.501,681,205,0.361,...,60,2120,32,104,1778,65,289,255,0,NaN
635,Cody Zeller,zelleco01,C,2019-20,CHO,233,0.577,404,18,0.240,...,39,1341,160,140,642,40,75,411,0,NaN
636,Ante Žižić,zizican01,C,2019-20,CLE,41,0.569,72,0,0.000,...,0,221,18,27,96,7,10,66,0,NaN


In [8]:
mvp_2019 = pd.merge(mvp_2019, mvp_votes, how='left', on=["Player", "Season"])
mvp_2019

,Player,PlayerID,Pos,Season,Tm,2P,2P%,2PA,3P,3P%,...,GS,MP,ORB,PF,PTS,STL,TOV,TRB,MVP,MVP_Candidate
0,Álex Abrines,abrinal01,SG,2018-19,OKC,15,0.500,30,41,0.323,...,2,588,5,53,165,17,14,48,0,NaN
1,Quincy Acy,acyqu01,PF,2018-19,PHO,2,0.667,3,2,0.133,...,0,123,3,24,17,1,4,25,0,NaN
2,Jaylen Adams,adamsja01,PG,2018-19,ATL,13,0.361,36,25,0.338,...,1,428,11,45,108,14,28,60,0,NaN
3,Steven Adams,adamsst01,C,2018-19,OKC,481,0.596,807,0,0.000,...,80,2669,391,204,1108,117,135,760,0,NaN
4,Bam Adebayo,adebaba01,C,2018-19,MIA,277,0.588,471,3,0.200,...,28,1913,165,203,729,71,121,597,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,Tyler Zeller,zellety01,C,2018-19,MEM,16,0.571,28,0,0.000,...,1,82,9,16,46,1,4,18,0,NaN
704,Ante Žižić,zizican01,C,2018-19,CLE,183,0.553,331,0,0.000,...,25,1082,108,113,459,13,61,320,0,NaN
705,Ivica Zubac,zubaciv01,C,2018-19,TOT,212,0.559,379,0,0.000,...,37,1040,115,137,525,14,70,362,0,NaN
706,Ivica Zubac,zubaciv01,C,2018-19,LAL,112,0.580,193,0,0.000,...,12,516,54,73,281,4,33,162,0,NaN


In [9]:
mvp_2018 = pd.merge(mvp_2018, mvp_votes, how='left', on=["Player", "Season"])
mvp_2018

,Player,PlayerID,Pos,Season,Tm,2P,2P%,2PA,3P,3P%,...,GS,MP,ORB,PF,PTS,STL,TOV,TRB,MVP,MVP_Candidate
0,Álex Abrines,abrinal01,SG,2017-18,OKC,31,0.443,70,84,0.380,...,8,1134,26,124,353,38,25,114,0,NaN
1,Quincy Acy,acyqu01,PF,2017-18,BRK,28,0.384,73,102,0.349,...,8,1359,40,149,411,33,60,257,0,NaN
2,Steven Adams,adamsst01,C,2017-18,OKC,448,0.631,710,0,0.000,...,76,2487,384,215,1056,92,128,685,0,NaN
3,Bam Adebayo,adebaba01,C,2017-18,MIA,174,0.523,333,0,0.000,...,19,1368,118,138,477,32,66,381,0,NaN
4,Arron Afflalo,afflaar01,SG,2017-18,ORL,38,0.413,92,27,0.386,...,3,682,4,56,179,4,21,66,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Tyler Zeller,zellety01,C,2017-18,BRK,115,0.567,203,10,0.385,...,33,703,63,78,300,8,35,194,0,NaN
660,Tyler Zeller,zellety01,C,2017-18,MIL,62,0.602,103,0,0.000,...,1,406,47,48,141,7,12,111,0,NaN
661,Paul Zipser,zipsepa01,SF,2017-18,CHI,44,0.355,124,37,0.336,...,12,824,13,86,218,20,43,131,0,NaN
662,Ante Žižić,zizican01,C,2017-18,CLE,49,0.731,67,0,0.000,...,2,214,24,30,119,2,11,60,0,NaN


In [10]:
mvp_2017 = pd.merge(mvp_2017, mvp_votes, how='left', on=["Player", "Season"])
mvp_2017

,Player,PlayerID,Pos,Season,Tm,2P,2P%,2PA,3P,3P%,...,GS,MP,ORB,PF,PTS,STL,TOV,TRB,MVP,MVP_Candidate
0,Álex Abrines,abrinal01,SG,2016-17,OKC,40,0.426,94,94,0.381,...,6,1055,18,114,406,37,33,86,0,NaN
1,Quincy Acy,acyqu01,PF,2016-17,TOT,33,0.413,80,37,0.411,...,1,558,20,67,222,14,21,115,0,NaN
2,Quincy Acy,acyqu01,PF,2016-17,DAL,4,0.400,10,1,0.143,...,0,48,2,9,13,0,2,8,0,NaN
3,Quincy Acy,acyqu01,PF,2016-17,BRK,29,0.414,70,36,0.434,...,1,510,18,58,209,14,19,107,0,NaN
4,Steven Adams,adamsst01,C,2016-17,OKC,374,0.572,654,0,0.000,...,80,2389,281,195,905,89,146,613,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,Cody Zeller,zelleco01,C,2016-17,CHO,253,0.572,442,0,0.000,...,58,1725,135,189,639,62,65,405,0,NaN
591,Tyler Zeller,zellety01,C,2016-17,BOS,78,0.497,157,0,0.000,...,5,525,43,61,178,7,20,124,0,NaN
592,Stephen Zimmerman,zimmest01,C,2016-17,ORL,10,0.323,31,0,0.000,...,0,108,11,17,23,2,3,35,0,NaN
593,Paul Zipser,zipsepa01,SF,2016-17,CHI,55,0.451,122,33,0.333,...,18,843,15,78,240,15,40,125,0,NaN


In [11]:
mvp_2017.columns

Index(['Player', 'PlayerID', 'Pos', 'Season', 'Tm', '2P', '2P%', '2PA', '3P',
       '3P%', '3PA', 'Age', 'AST', 'BLK', 'DRB', 'eFG%', 'FG', 'FG%', 'FGA',
       'FT', 'FT%', 'FTA', 'G', 'GS', 'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV',
       'TRB', 'MVP', 'MVP_Candidate'],
      dtype='object')

In [12]:
# from sklearn import neighbors
# from math import sqrt
# from sklearn.metrics import mean_squared_error 

# rmse_val = [] #to store rmse values for different k
# for K in range(20):
#     K = K+1
#     model = neighbors.KNeighborsRegressor(n_neighbors = K)

#     model.fit(X_train, y_train)  #fit the model
#     pred=model.predict(X_test) #make prediction on test set
#     error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
#     rmse_val.append(error) #store rmse values
#     print('RMSE value for k= ' , K , 'is:', error)

In [13]:
# curve = pd.DataFrame(rmse_val) 
# curve.plot()

# ## 9 appears to be the best value for k

In [14]:
# plt.plot(range(1, 20, 2), train_scores, marker='o')
# plt.plot(range(1, 20, 2), test_scores, marker="x")
# plt.xlabel("k neighbors")
# plt.ylabel("Testing accuracy Score")
# plt.show()

In [15]:
# knn = KNeighborsClassifier(n_neighbors=9)
# knn.fit(X_train_scaled, y_train)
# print('k=9 Test Acc: %.4f' % knn.score(X_test_scaled, y_test))

In [16]:
# from sklearn.model_selection import GridSearchCV
# from sklearn import neighbors
# params = {'n_neighbors':[2,3,4,5,6,7,8,9,10]}

# knn = neighbors.KNeighborsRegressor()

# model = GridSearchCV(knn, params, cv=5)
# model.fit(X_train,y_train)
# model.best_params_

In [17]:
X = mvp[["PTS", "", "Pos", "Season", "Tm", "MVP","MVP_Candidate"]]
Xx = mvp_2020[["Player", "PlayerID", "Pos", "Season", "Tm", "MVP", "MVP_Candidate"]]
X_2019 = mvp_2019[["Player", "PlayerID", "Pos", "Season", "Tm", "MVP", "MVP_Candidate"]]
X_2018 = mvp_2018[["Player", "PlayerID", "Pos", "Season", "Tm", "MVP", "MVP_Candidate"]]
X_2017 = mvp_2017[["Player", "PlayerID", "Pos", "Season", "Tm", "MVP", "MVP_Candidate"]]
y = mvp["MVP_Candidate"]
y = y.values.reshape(-1,1)

In [18]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler

X_scaler1 = StandardScaler().fit(X_train1)
y_scaler1 = StandardScaler().fit(y_train1)

In [20]:
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)
Xx_test_scaled = X_scaler1.transform(Xx)
X_2019_test_scaled = X_scaler1.transform(X_2019)
X_2018_test_scaled = X_scaler1.transform(X_2018)
X_2017_test_scaled = X_scaler1.transform(X_2017)
y_train_scaled1 = y_scaler1.transform(y_train1)
y_test_scaled1 = y_scaler1.transform(y_test1)

In [21]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_scaler_rf = MinMaxScaler().fit(X_train1)
X_train_scaled_rf = X_scaler1.transform(X_train1)
X_test_scaled_rf = X_scaler1.transform(X_test1)
Xx_test_scaled_rf = X_scaler1.transform(Xx)
X_2019_test_scaled_rf = X_scaler1.transform(X_2019)
X_2018_test_scaled_rf = X_scaler1.transform(X_2018)
X_2017_test_scaled_rf = X_scaler1.transform(X_2017)

label_encoder = LabelEncoder()
label_encoder.fit(y_train1)
encoded_y_train = label_encoder.transform(y_train1)
encoded_y_test = label_encoder.transform(y_test1)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled_rf, encoded_y_train)

print(f"Training Data Score: {rf.score(X_train_scaled_rf, encoded_y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled_rf, encoded_y_test)}")

C:\Users\ricks\AppData\Local\Continuum\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training Data Score: 1.0
Testing Data Score: 0.9898916967509025


In [22]:
from sklearn.svm import SVC 

from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [400, 700, 1000]
}
grid2 = GridSearchCV(estimator=rf, param_grid=param_grid, verbose=3)

grid2.fit(X_train_scaled_rf, encoded_y_train)

print(grid2.best_params_)
print(grid2.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] n_estimators=400 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... n_estimators=400, score=0.986, total=   8.4s
[CV] n_estimators=400 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.3s remaining:    0.0s


[CV] .................... n_estimators=400, score=0.983, total=   7.7s
[CV] n_estimators=400 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.0s remaining:    0.0s


[CV] .................... n_estimators=400, score=0.986, total=   7.2s
[CV] n_estimators=400 ................................................
[CV] .................... n_estimators=400, score=0.988, total=   7.3s
[CV] n_estimators=400 ................................................
[CV] .................... n_estimators=400, score=0.987, total=   6.7s
[CV] n_estimators=700 ................................................
[CV] .................... n_estimators=700, score=0.986, total=  12.5s
[CV] n_estimators=700 ................................................
[CV] .................... n_estimators=700, score=0.983, total=  12.4s
[CV] n_estimators=700 ................................................
[CV] .................... n_estimators=700, score=0.984, total=  12.4s
[CV] n_estimators=700 ................................................
[CV] .................... n_estimators=700, score=0.989, total=  12.3s
[CV] n_estimators=700 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.1min finished


{'n_estimators': 700}
0.98604091456077


In [23]:
# SVC Prediction for 2017
predictions_2017 = grid2.predict(X_2017_test_scaled_rf)

# SVC Prediction for 2018
predictions_2018 = grid2.predict(X_2018_test_scaled_rf)

# SVC Prediction for 2019
predictions1 = grid2.predict(X_2019_test_scaled_rf)

# SVC Prediction for 2020
predictions2 = grid2.predict(Xx_test_scaled_rf)

In [24]:
names_list = []
season_list = []
team_list = []
model_list = []

In [25]:
# 2017
found = 0
for i in range(len(predictions_2017)):
    if predictions_2017[i] == 1:
        found = 1
        print(i,mvp_2017.iloc[i,0], mvp_2017.iloc[i,3], mvp_2017.iloc[i,4])
        names_list.append(mvp_2017.iloc[i,0])
        season_list.append(mvp_2017.iloc[i,3])
        team_list.append(mvp_2017.iloc[i,4])
        model_list.append("Base Stats Random Forest")
if found == 0:
    print("No Candidates")

19 Giannis Antetokounmpo 2016-17 MIL
120 Stephen Curry 2016-17 GSW
122 Anthony Davis 2016-17 NOP
210 James Harden 2016-17 HOU
269 LeBron James 2016-17 CLE
318 Kawhi Leonard 2016-17 SAS
516 Isaiah Thomas 2016-17 BOS
550 John Wall 2016-17 WAS
558 Russell Westbrook 2016-17 OKC


In [26]:
# 2018
found = 0
for i in range(len(predictions_2018)):
    if predictions_2018[i] == 1:
        found = 1
        print(i,mvp_2018.iloc[i,0], mvp_2018.iloc[i,3], mvp_2018.iloc[i,4])
        names_list.append(mvp_2018.iloc[i,0])
        season_list.append(mvp_2018.iloc[i,3])
        team_list.append(mvp_2018.iloc[i,4])
        model_list.append("Base Stats Random Forest")
if found == 0:
    print("No Candidates")

6 LaMarcus Aldridge 2017-18 SAS
15 Giannis Antetokounmpo 2017-18 MIL
93 Jimmy Butler 2017-18 MIN
146 Stephen Curry 2017-18 GSW
148 Anthony Davis 2017-18 NOP
156 DeMar DeRozan 2017-18 TOR
171 Kevin Durant 2017-18 GSW
177 Joel Embiid 2017-18 PHI
232 James Harden 2017-18 HOU
303 LeBron James 2017-18 CLE
369 Damian Lillard 2017-18 POR
472 Victor Oladipo 2017-18 IND
625 Russell Westbrook 2017-18 OKC


In [27]:
# 2019
found = 0
for i in range(len(predictions1)):
    if predictions1[i] == 1:
        found = 1
        print(i,mvp_2019.iloc[i,0], mvp_2019.iloc[i,3], mvp_2019.iloc[i,4])
        names_list.append(mvp_2019.iloc[i,0])
        season_list.append(mvp_2019.iloc[i,3])
        team_list.append(mvp_2019.iloc[i,4])
        model_list.append("Base Stats Random Forest")
if found == 0:
    print("No Candidates")

19 Giannis Antetokounmpo 2018-19 MIL
159 Stephen Curry 2018-19 GSW
191 Kevin Durant 2018-19 GSW
200 Joel Embiid 2018-19 PHI
240 Paul George 2018-19 OKC
261 Blake Griffin 2018-19 DET
267 James Harden 2018-19 HOU
332 LeBron James 2018-19 LAL
397 Kawhi Leonard 2018-19 TOR
401 Damian Lillard 2018-19 POR
675 Russell Westbrook 2018-19 OKC


In [28]:
# 2020
found = 0
for i in range(len(predictions2)):
    if predictions2[i] == 1:
        found = 1
        print(i,mvp_2020.iloc[i,0], mvp_2020.iloc[i,3], mvp_2020.iloc[i,4])
        names_list.append(mvp_2020.iloc[i,0])
        season_list.append(mvp_2020.iloc[i,3])
        team_list.append(mvp_2020.iloc[i,4])
        model_list.append("Base Stats Random Forest")
if found == 0:
    print("No Candidates")

11 Giannis Antetokounmpo 2019-20 MIL
250 James Harden 2019-20 HOU


In [29]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled_rf, encoded_y_train)
    train_score = knn.score(X_train_scaled_rf, encoded_y_train)
    test_score = knn.score(X_test_scaled_rf, encoded_y_test)
    #knn.fit(X_train_scaled, encoded_y_train)
    #train_score = knn.score(X_train_scaled,encoded_y_train)
    #test_score = knn.score(X_test_scaled, encoded_y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled_rf, encoded_y_train)
print('Test Acc: %.3f' % knn.score(X_test_scaled_rf, encoded_y_test))
print(f"Training Data Score: {knn.score(X_train_scaled_rf, encoded_y_train)}")
print(f"Testing Data Score: {knn.score(X_test_scaled_rf, encoded_y_test)}")

k: 1, Train/Test Score: 1.000/0.982
k: 3, Train/Test Score: 0.991/0.987
k: 5, Train/Test Score: 0.990/0.989
k: 7, Train/Test Score: 0.988/0.991
k: 9, Train/Test Score: 0.987/0.990
k: 11, Train/Test Score: 0.987/0.989
k: 13, Train/Test Score: 0.986/0.989
k: 15, Train/Test Score: 0.986/0.988
k: 17, Train/Test Score: 0.985/0.988
k: 19, Train/Test Score: 0.985/0.987
Test Acc: 0.988
Training Data Score: 0.9855595667870036
Testing Data Score: 0.9884476534296028


In [30]:
#2017
predictions3 = knn.predict(X_2017_test_scaled_rf)
#2018
predictions4 = knn.predict(X_2018_test_scaled_rf)
#2019
predictions1 = knn.predict(X_2019_test_scaled_rf)
#2020
predictions2 = knn.predict(Xx_test_scaled_rf)

In [31]:
for i in range(len(predictions3)):
    if predictions3[i] == 1:
        print(i,mvp_2017.iloc[i,0],mvp_2017.iloc[i,3],mvp_2017.iloc[i,4])
        names_list.append(mvp_2017.iloc[i,0])
        season_list.append(mvp_2017.iloc[i,3])
        team_list.append(mvp_2017.iloc[i,4])
        model_list.append("Base Stats KNN")

120 Stephen Curry 2016-17 GSW
122 Anthony Davis 2016-17 NOP
210 James Harden 2016-17 HOU
269 LeBron James 2016-17 CLE
558 Russell Westbrook 2016-17 OKC


In [32]:
for i in range(len(predictions4)):
    if predictions4[i] == 1:
        print(i,mvp_2018.iloc[i,0], mvp_2018.iloc[i,3], mvp_2018.iloc[i,4])
        names_list.append(mvp_2018.iloc[i,0])
        season_list.append(mvp_2018.iloc[i,3])
        team_list.append(mvp_2018.iloc[i,4])
        model_list.append("Base Stats KNN")

148 Anthony Davis 2017-18 NOP
232 James Harden 2017-18 HOU
303 LeBron James 2017-18 CLE
625 Russell Westbrook 2017-18 OKC


In [33]:
for i in range(len(predictions1)):
    if predictions1[i] == 1:
        print(i,mvp_2019.iloc[i,0], mvp_2019.iloc[i,3], mvp_2019.iloc[i,4])
        names_list.append(mvp_2019.iloc[i,0])
        season_list.append(mvp_2019.iloc[i,3])
        team_list.append(mvp_2019.iloc[i,4])
        model_list.append("Base Stats KNN")

191 Kevin Durant 2018-19 GSW
200 Joel Embiid 2018-19 PHI
267 James Harden 2018-19 HOU
675 Russell Westbrook 2018-19 OKC


In [34]:
for i in range(len(predictions2)):
    if predictions2[i] == 1:
        print(i,mvp_2020.iloc[i,0], mvp_2020.iloc[i,3], mvp_2020.iloc[i,4])
        names_list.append(mvp_2020.iloc[i,0])
        season_list.append(mvp_2020.iloc[i,3])
        team_list.append(mvp_2020.iloc[i,4])
        model_list.append("Base Stats KNN")

11 Giannis Antetokounmpo 2019-20 MIL
250 James Harden 2019-20 HOU


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=8, activation='relu', input_dim=26))
deep_model.add(Dense(units=8, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

deep_model.summary()

deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled_rf,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)


model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled_rf,y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 216       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 306
Trainable params: 306
Non-trainable params: 0
_________________________________________________________________
Train on 8310 samples
Epoch 1/1000
8310/8310 - 1s - loss: 0.1524 - accuracy: 0.9812
Epoch 2/1000
8310/8310 - 1s - loss: 0.0588 - accuracy: 0.9812
Epoch 3/1000
8310/8310 - 0s - loss: 0.0519 - accuracy: 0.9812
Epoch 4/1000
8310/8310 - 0s - loss: 0.0484 - accuracy: 0.9812
Epoch 5/1000
8310/8310 - 0s - loss: 0.0458 - accuracy: 0.9812
Epoch 6/1000
8310/8310 - 0s - loss: 0.0438 - accuracy: 0

Epoch 119/1000
8310/8310 - 0s - loss: 0.0252 - accuracy: 0.9889
Epoch 120/1000
8310/8310 - 0s - loss: 0.0256 - accuracy: 0.9892
Epoch 121/1000
8310/8310 - 0s - loss: 0.0251 - accuracy: 0.9883
Epoch 122/1000
8310/8310 - 0s - loss: 0.0249 - accuracy: 0.9886
Epoch 123/1000
8310/8310 - 0s - loss: 0.0250 - accuracy: 0.9886
Epoch 124/1000
8310/8310 - 0s - loss: 0.0250 - accuracy: 0.9889
Epoch 125/1000
8310/8310 - 0s - loss: 0.0244 - accuracy: 0.9889
Epoch 126/1000
8310/8310 - 0s - loss: 0.0248 - accuracy: 0.9892
Epoch 127/1000
8310/8310 - 0s - loss: 0.0245 - accuracy: 0.9889
Epoch 128/1000
8310/8310 - 0s - loss: 0.0247 - accuracy: 0.9890
Epoch 129/1000
8310/8310 - 0s - loss: 0.0247 - accuracy: 0.9886
Epoch 130/1000
8310/8310 - 0s - loss: 0.0242 - accuracy: 0.9895
Epoch 131/1000
8310/8310 - 0s - loss: 0.0242 - accuracy: 0.9894
Epoch 132/1000
8310/8310 - 0s - loss: 0.0246 - accuracy: 0.9895
Epoch 133/1000
8310/8310 - 0s - loss: 0.0245 - accuracy: 0.9888
Epoch 134/1000
8310/8310 - 0s - loss: 0.

8310/8310 - 0s - loss: 0.0192 - accuracy: 0.9913
Epoch 248/1000
8310/8310 - 0s - loss: 0.0204 - accuracy: 0.9913
Epoch 249/1000
8310/8310 - 0s - loss: 0.0198 - accuracy: 0.9912
Epoch 250/1000
8310/8310 - 0s - loss: 0.0206 - accuracy: 0.9912
Epoch 251/1000
8310/8310 - 0s - loss: 0.0196 - accuracy: 0.9911
Epoch 252/1000
8310/8310 - 0s - loss: 0.0204 - accuracy: 0.9916
Epoch 253/1000
8310/8310 - 0s - loss: 0.0188 - accuracy: 0.9918
Epoch 254/1000
8310/8310 - 0s - loss: 0.0195 - accuracy: 0.9915
Epoch 255/1000
8310/8310 - 0s - loss: 0.0197 - accuracy: 0.9918
Epoch 256/1000
8310/8310 - 0s - loss: 0.0197 - accuracy: 0.9916
Epoch 257/1000
8310/8310 - 0s - loss: 0.0201 - accuracy: 0.9912
Epoch 258/1000
8310/8310 - 0s - loss: 0.0196 - accuracy: 0.9913
Epoch 259/1000
8310/8310 - 0s - loss: 0.0197 - accuracy: 0.9911
Epoch 260/1000
8310/8310 - 0s - loss: 0.0193 - accuracy: 0.9910
Epoch 261/1000
8310/8310 - 0s - loss: 0.0193 - accuracy: 0.9915
Epoch 262/1000
8310/8310 - 0s - loss: 0.0205 - accuracy

Epoch 376/1000
8310/8310 - 0s - loss: 0.0173 - accuracy: 0.9928
Epoch 377/1000
8310/8310 - 0s - loss: 0.0176 - accuracy: 0.9928
Epoch 378/1000
8310/8310 - 0s - loss: 0.0167 - accuracy: 0.9921
Epoch 379/1000
8310/8310 - 0s - loss: 0.0187 - accuracy: 0.9921
Epoch 380/1000
8310/8310 - 0s - loss: 0.0171 - accuracy: 0.9928
Epoch 381/1000
8310/8310 - 0s - loss: 0.0181 - accuracy: 0.9919
Epoch 382/1000
8310/8310 - 0s - loss: 0.0178 - accuracy: 0.9927
Epoch 383/1000
8310/8310 - 0s - loss: 0.0169 - accuracy: 0.9922
Epoch 384/1000
8310/8310 - 0s - loss: 0.0179 - accuracy: 0.9917
Epoch 385/1000
8310/8310 - 0s - loss: 0.0173 - accuracy: 0.9923
Epoch 386/1000
8310/8310 - 0s - loss: 0.0168 - accuracy: 0.9923
Epoch 387/1000
8310/8310 - 0s - loss: 0.0182 - accuracy: 0.9922
Epoch 388/1000
8310/8310 - 0s - loss: 0.0169 - accuracy: 0.9924
Epoch 389/1000
8310/8310 - 0s - loss: 0.0179 - accuracy: 0.9921
Epoch 390/1000
8310/8310 - 0s - loss: 0.0186 - accuracy: 0.9922
Epoch 391/1000
8310/8310 - 0s - loss: 0.

8310/8310 - 0s - loss: 0.0147 - accuracy: 0.9930
Epoch 505/1000
8310/8310 - 0s - loss: 0.0170 - accuracy: 0.9925
Epoch 506/1000
8310/8310 - 0s - loss: 0.0159 - accuracy: 0.9925
Epoch 507/1000
8310/8310 - 0s - loss: 0.0156 - accuracy: 0.9930
Epoch 508/1000
8310/8310 - 0s - loss: 0.0146 - accuracy: 0.9940
Epoch 509/1000
8310/8310 - 0s - loss: 0.0170 - accuracy: 0.9927
Epoch 510/1000
8310/8310 - 0s - loss: 0.0138 - accuracy: 0.9940
Epoch 511/1000
8310/8310 - 0s - loss: 0.0149 - accuracy: 0.9934
Epoch 512/1000
8310/8310 - 0s - loss: 0.0146 - accuracy: 0.9933
Epoch 513/1000
8310/8310 - 0s - loss: 0.0142 - accuracy: 0.9939
Epoch 514/1000
8310/8310 - 0s - loss: 0.0160 - accuracy: 0.9935
Epoch 515/1000
8310/8310 - 0s - loss: 0.0156 - accuracy: 0.9927
Epoch 516/1000
8310/8310 - 0s - loss: 0.0147 - accuracy: 0.9933
Epoch 517/1000
8310/8310 - 0s - loss: 0.0144 - accuracy: 0.9936
Epoch 518/1000
8310/8310 - 0s - loss: 0.0152 - accuracy: 0.9940
Epoch 519/1000
8310/8310 - 0s - loss: 0.0144 - accuracy

Epoch 633/1000
8310/8310 - 1s - loss: 0.0133 - accuracy: 0.9946
Epoch 634/1000
8310/8310 - 1s - loss: 0.0140 - accuracy: 0.9940
Epoch 635/1000
8310/8310 - 1s - loss: 0.0132 - accuracy: 0.9943
Epoch 636/1000
8310/8310 - 0s - loss: 0.0136 - accuracy: 0.9946
Epoch 637/1000
8310/8310 - 0s - loss: 0.0136 - accuracy: 0.9948
Epoch 638/1000
8310/8310 - 0s - loss: 0.0151 - accuracy: 0.9930
Epoch 639/1000
8310/8310 - 1s - loss: 0.0133 - accuracy: 0.9946
Epoch 640/1000
8310/8310 - 0s - loss: 0.0140 - accuracy: 0.9935
Epoch 641/1000
8310/8310 - 0s - loss: 0.0142 - accuracy: 0.9939
Epoch 642/1000
8310/8310 - 0s - loss: 0.0130 - accuracy: 0.9946
Epoch 643/1000
8310/8310 - 0s - loss: 0.0139 - accuracy: 0.9941
Epoch 644/1000
8310/8310 - 0s - loss: 0.0151 - accuracy: 0.9939
Epoch 645/1000
8310/8310 - 0s - loss: 0.0144 - accuracy: 0.9928
Epoch 646/1000
8310/8310 - 0s - loss: 0.0132 - accuracy: 0.9939
Epoch 647/1000
8310/8310 - 0s - loss: 0.0148 - accuracy: 0.9925
Epoch 648/1000
8310/8310 - 0s - loss: 0.

8310/8310 - 0s - loss: 0.0127 - accuracy: 0.9945
Epoch 762/1000
8310/8310 - 0s - loss: 0.0134 - accuracy: 0.9949
Epoch 763/1000
8310/8310 - 0s - loss: 0.0125 - accuracy: 0.9949
Epoch 764/1000
8310/8310 - 0s - loss: 0.0115 - accuracy: 0.9958
Epoch 765/1000
8310/8310 - 0s - loss: 0.0148 - accuracy: 0.9937
Epoch 766/1000
8310/8310 - 0s - loss: 0.0155 - accuracy: 0.9930
Epoch 767/1000
8310/8310 - 0s - loss: 0.0132 - accuracy: 0.9942
Epoch 768/1000
8310/8310 - 0s - loss: 0.0123 - accuracy: 0.9951
Epoch 769/1000
8310/8310 - 0s - loss: 0.0123 - accuracy: 0.9948
Epoch 770/1000
8310/8310 - 0s - loss: 0.0121 - accuracy: 0.9947
Epoch 771/1000
8310/8310 - 0s - loss: 0.0146 - accuracy: 0.9935
Epoch 772/1000
8310/8310 - 0s - loss: 0.0148 - accuracy: 0.9935
Epoch 773/1000
8310/8310 - 0s - loss: 0.0129 - accuracy: 0.9942
Epoch 774/1000
8310/8310 - 0s - loss: 0.0128 - accuracy: 0.9947
Epoch 775/1000
8310/8310 - 0s - loss: 0.0134 - accuracy: 0.9940
Epoch 776/1000
8310/8310 - 0s - loss: 0.0125 - accuracy

Epoch 890/1000
8310/8310 - 0s - loss: 0.0138 - accuracy: 0.9948
Epoch 891/1000
8310/8310 - 0s - loss: 0.0123 - accuracy: 0.9943
Epoch 892/1000
8310/8310 - 0s - loss: 0.0129 - accuracy: 0.9946
Epoch 893/1000
8310/8310 - 0s - loss: 0.0135 - accuracy: 0.9942
Epoch 894/1000
8310/8310 - 0s - loss: 0.0149 - accuracy: 0.9934
Epoch 895/1000
8310/8310 - 0s - loss: 0.0119 - accuracy: 0.9951
Epoch 896/1000
8310/8310 - 0s - loss: 0.0115 - accuracy: 0.9955
Epoch 897/1000
8310/8310 - 0s - loss: 0.0115 - accuracy: 0.9951
Epoch 898/1000
8310/8310 - 0s - loss: 0.0123 - accuracy: 0.9949
Epoch 899/1000
8310/8310 - 0s - loss: 0.0113 - accuracy: 0.9948
Epoch 900/1000
8310/8310 - 0s - loss: 0.0154 - accuracy: 0.9946
Epoch 901/1000
8310/8310 - 0s - loss: 0.0119 - accuracy: 0.9942
Epoch 902/1000
8310/8310 - 0s - loss: 0.0113 - accuracy: 0.9953
Epoch 903/1000
8310/8310 - 0s - loss: 0.0117 - accuracy: 0.9953
Epoch 904/1000
8310/8310 - 0s - loss: 0.0119 - accuracy: 0.9949
Epoch 905/1000
8310/8310 - 0s - loss: 0.

In [38]:
ynew3 = deep_model.predict_classes(X_2017_test_scaled_rf)
for i in range(len(X_2017_test_scaled_rf)):
    if (ynew3[i] == 1):
        print(i,"X=%s, Predicted=%s" % (X_2017_test_scaled_rf[i], ynew3[i]),mvp_2017.iloc[i,0])
        names_list.append(mvp_2017.iloc[i,0])
        season_list.append(mvp_2017.iloc[i,3])
        team_list.append(mvp_2017.iloc[i,4])
        model_list.append("Base Stats Deep Learning")

19 X=[ 3.30308259  0.9435178   2.77094881  0.36370237  0.18746901  0.7318039
 -1.11464233  2.66758101  4.08746802  3.01406358  0.65824593  2.93304368
  0.87608266  2.4683409   3.89856778  0.36951238  3.97249306  1.23898421
  2.08154722  1.96271685  1.5844951   2.06959946  3.04621846  2.89514706
  2.83487512  2.67667026], Predicted=1 Giannis Antetokounmpo
105 X=[ 1.10246574  0.36540274  1.0715414   3.02149409  0.94511618  2.69490949
  0.51402171  2.65961026 -0.09022884  0.49517252  0.6943396   1.77906582
  0.29561632  1.7721922   2.34644212  0.78491072  2.04958893  0.82144609
  1.64784443  1.34114563 -0.32872919  0.41632056  2.12707835  1.72564036
  1.54328664  0.25979612], Predicted=1 Mike Conley
120 X=[ 1.50894911  0.71577551  1.29395873  6.35462633  0.96182899  5.73402694
  0.28135542  3.38494909 -0.15352727  1.08870432  1.01015927  3.0472394
  0.37174305  2.98070634  2.43656555  0.96693921  2.0023044   1.2010262
  2.04540532  1.73004914  0.18835846  1.20162804  3.41431535  3.2250079

In [39]:
ynew2 = deep_model.predict_classes(X_2018_test_scaled_rf)
for i in range(len(X_2018_test_scaled_rf)):
    if (ynew2[i] == 1):
        print(i,"X=%s, Predicted=%s" % (X_2018_test_scaled_rf[i], ynew2[i]),mvp_2018.iloc[i,0])
        names_list.append(mvp_2018.iloc[i,0])
        season_list.append(mvp_2018.iloc[i,3])
        team_list.append(mvp_2018.iloc[i,4])
        model_list.append("Base Stats Deep Learning")

6 X=[ 3.67452429  0.619423    3.38259646 -0.11557155  0.30445864  0.00898678
  1.21202059  0.4198277   2.15686568  1.79080901  0.46875413  3.11936302
  0.7714084   2.71338524  2.52668897  0.68222799  2.29389232  1.04919416
  1.90083773  1.58505333  3.37706559  0.89852691  2.83241369  0.25626014
  0.79813943  2.3344114 ], Predicted=1 LaMarcus Aldridge
15 X=[ 3.94784931  0.86468393  3.40692336  0.2329913   0.38245174  0.40325067
 -0.88197604  2.08571579  2.66325318  3.2963531   0.6943396   3.44992959
  0.95220939  2.86653795  4.0587872   0.32283841  4.20103495  1.04919416
  1.90083773  1.86268097  1.82580266  1.8629396   3.44737794  2.23542533
  2.65272803  2.95574287], Predicted=1 Giannis Antetokounmpo
93 X=[ 1.62108245  0.47051457  1.52680187  0.82119111  0.62200195  0.89608052
  0.28135542  1.50385058  0.06801726  0.67612733  0.39656678  1.61678769
  0.4288381   1.52714785  2.81708668  0.76157374  2.49879194  0.44186597
  1.32256734  1.19727382  0.49861104 -0.244991    1.88902767  2.4

In [40]:
ynew1 = deep_model.predict_classes(X_2019_test_scaled_rf)
for i in range(len(X_2019_test_scaled_rf)):
    if (ynew1[i] == 1):
        print(i,"X=%s, Predicted=%s" % (X_2019_test_scaled_rf[i], ynew1[i]),mvp_2019.iloc[i,0])
        names_list.append(mvp_2019.iloc[i,0])
        season_list.append(mvp_2019.iloc[i,3])
        team_list.append(mvp_2019.iloc[i,4])
        model_list.append("Base Stats Deep Learning")

19 X=[ 3.73759929  1.62674469  2.64931433  0.4290579   0.09833404  0.92072201
 -0.64930974  2.58787345  2.78985006  4.32417647  1.18160423  3.32371326
  1.41848564  2.43492576  4.18896548  0.1781491   4.55566891  0.93532012
  1.79241203  1.41532954  1.87751143  1.87671692  3.40329448  1.72564036
  3.39787523  3.71924342], Predicted=1 Giannis Antetokounmpo
143 X=[ 1.39681577  0.24277228  1.43296956  2.67293124  0.69999504  2.75240631
  0.9793543   2.78714236  0.00471882  0.4155524   0.35144968  1.93533366
  0.0862678   2.08128222  2.6168124   0.71956716  2.3490576   0.8594041
  1.72012823  1.39734556 -0.17360289  0.37498859  2.26594124  1.78561506
  1.11275714  0.24926508], Predicted=1 Mike Conley
159 X=[ 0.99734073  0.61066368  0.86302516  7.00818167  1.1066733   5.90651739
  0.74668801  2.08571579  0.09966647  1.32032649  1.22672132  2.78879645
  0.40980642  2.69389307  1.81571529  1.05095236  1.4112478   0.82144609
  1.68398633  1.38498158 -0.08742162  0.96741353  3.15422294  1.72564

In [41]:
ynew = deep_model.predict_classes(Xx_test_scaled_rf)
for i in range(len(Xx_test_scaled_rf)):
    if (ynew[i] == 1):
        print(i,"X=%s, Predicted=%s" % (Xx_test_scaled_rf[i], ynew[i]),mvp_2020.iloc[i,0])
        names_list.append(mvp_2020.iloc[i,0])
        season_list.append(mvp_2020.iloc[i,3])
        team_list.append(mvp_2020.iloc[i,4])
        model_list.append("Base Stats Deep Learning")

11 X=[ 2.92463256  1.48659559  2.09674628  1.1261836   0.38245174  1.50390401
 -0.41664345  1.88644688  1.17573991  3.80302659  1.06429978  2.81884795
  1.15204207  2.18988142  2.86715525 -0.27458842  3.73607042  0.40390796
  1.28642544  0.78251834  1.39489629  1.13274142  2.81257613  0.76604511
  2.45402211  3.19269132], Predicted=1 Giannis Antetokounmpo
155 X=[ 2.19576583  0.84716529  1.85000205  0.75583557  0.54957979  0.88786669
 -0.18397716  0.61112586  3.5494313   1.86319094  0.66726935  2.09160149
  0.76189256  1.78333058  3.20762152  0.73356935  2.88494892  0.32799193
  1.21414165  0.92863816  1.30871502  0.63675775  2.31663722  1.42576684
  1.26178658  1.75520408], Predicted=1 Anthony Davis
162 X=[ 2.58823253  0.72453483  2.2844109  -0.52948993  0.15961433 -0.50027074
  0.74668801  1.99803747 -0.24847493  1.2117536   0.59508199  2.07357059
  0.95220939  1.6608084   2.63683983  0.70089757  2.38058062  0.55574
  1.43099304  1.15793387 -0.17360289  0.99496818  2.04772812  0.82601

In [42]:
df = pd.DataFrame({'Player': names_list, 'season': season_list, 'Tm': team_list, 'model': model_list})
df.to_sql(name='mvp_predictions', con=engine, if_exists='append', index=False)